In [1]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Загрузка данных
def load_data():
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    return (x_train, y_train), (x_test, y_test)

In [3]:
# Предобработка данных
def preprocess_data(x_train, x_test, y_train, y_test):
    x_train_normalized = x_train.reshape((-1, 28, 28, 1)) / 255.0
    x_test_normalized = x_test.reshape((-1, 28, 28, 1)) / 255.0
    y_train_encoded = to_categorical(y_train, num_classes=10)
    y_test_encoded = to_categorical(y_test, num_classes=10)
    return x_train_normalized, x_test_normalized, y_train_encoded, y_test_encoded

In [4]:
# Создание модели
def build_model():
    return Sequential(
        [
            Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
            Conv2D(64, kernel_size=(3, 3), activation="relu"),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),  # регуляризация модели
            Flatten(),
            Dense(128, activation="relu"),
            Dropout(0.5),
            Dense(10, activation="softmax"),
        ]
    )

In [5]:
# Компиляция модели
def compile_model(model):
    model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

In [6]:
# Аугментация данных
def augment_data():
    return ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
    )

In [7]:
# Обучение модели
def train_model(model, x_train, y_train, datagen, epochs=20, batch_size=128, x_test=None, y_test=None):
    steps_per_epoch = len(x_train) / batch_size
    validation_data = (x_test, y_test) if x_test is not None and y_test is not None else None
    model.fit(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_data,
    )

In [8]:
# Оценка модели
def evaluate_model(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Потери на тестовой выборке:", score[0])
    print("Точность на тестовой выборке:", score[1])

In [9]:
# Генерация отчета о классификации
def generate_classification_report(model, x_test, y_test):
    y_pred_probabilities = model.predict(x_test)
    y_pred = np.argmax(y_pred_probabilities, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    class_report = classification_report(y_test_classes, y_pred)
    print("Отчет о классификации:")
    print(class_report)

In [10]:
# Загрузка данных
(x_train, y_train), (x_test, y_test) = load_data()

# Предобработка данных
x_train_processed, x_test_processed, y_train_encoded, y_test_encoded = preprocess_data(x_train, x_test, y_train, y_test)

# Создание модели
model = build_model()

# Компиляция модели
compile_model(model)

# Аугментация данных
data_augmentor = augment_data()

# Обучение модели
train_model(
    model,
    x_train_processed,
    y_train_encoded,
    data_augmentor,
    epochs=20,
    batch_size=128,
    x_test=x_test_processed,
    y_test=y_test_encoded,
)

# Оценка модели
evaluate_model(model, x_test_processed, y_test_encoded)

# Генерация отчета о классификации
generate_classification_report(model, x_test_processed, y_test_encoded)

Epoch 1/20
468/468 [==============================] - 91s 191ms/step - loss: 0.8111 - accuracy: 0.7028 - val_loss: 0.4974 - val_accuracy: 0.8042
Epoch 2/20
468/468 [==============================] - 89s 191ms/step - loss: 0.5956 - accuracy: 0.7785 - val_loss: 0.4421 - val_accuracy: 0.8278
Epoch 3/20
468/468 [==============================] - 89s 189ms/step - loss: 0.5365 - accuracy: 0.8002 - val_loss: 0.3832 - val_accuracy: 0.8530
Epoch 4/20
468/468 [==============================] - 89s 189ms/step - loss: 0.5003 - accuracy: 0.8138 - val_loss: 0.3728 - val_accuracy: 0.8603
Epoch 5/20
468/468 [==============================] - 89s 191ms/step - loss: 0.4735 - accuracy: 0.8246 - val_loss: 0.3418 - val_accuracy: 0.8737
Epoch 6/20
468/468 [==============================] - 93s 198ms/step - loss: 0.4527 - accuracy: 0.8326 - val_loss: 0.3307 - val_accuracy: 0.8821
Epoch 7/20
468/468 [==============================] - 91s 195ms/step - loss: 0.4364 - accuracy: 0.8372 - val_loss: 0.3266 - val_ac

В результате экспериментов было установлено, что архитектура сверточной нейронной сети, чередующаяся с полносвязными слоями, продемонстрировала хорошие результаты.  
_Dropout(0.25)_ применяется для регуляризации модели и предотвращения переобучения. Этот слой случайным образом отключает указанную долю нейронов во время обучения, что помогает сети избежать слишком сильной зависимости между нейронами и улучшить ее обобщающую способность.  
Аугментация данных была применена с целью улучшения обобщающей способности модели и снижения риска переобучения.  
Рекомендуется ограничить количество эпох обучения сети не более 20, так как после этого момента эффективность обучения существенно снижается.  

Сеть демонстрирует лучшие результаты, однако время обучения заметно увеличивается. Работа проводилось на локальном компьютере.